In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio
import os
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# IMPORTANT: Remember to create a .env variable containing: OPENAI_API_KEY=sk-xyz where xyz is your key

# Access the API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAI API client
openai.api_key = api_key

nest_asyncio.apply()

In [4]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../data/Robinson_Advisory.txt", encoding="windows-1252")
index = VectorstoreIndexCreator().from_loaders([loader])


text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=0
)
documents = loader.load_and_split()
# texts = text_splitter.split_text(document)
docs = text_splitter.split_documents(documents)

In [5]:
embeddings = OpenAIEmbeddings()

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name="contract",
    connection_args={"host": "127.0.0.1", "port": "19530", "database": "contract"},
)
vector_db = Milvus(
    embeddings,
    connection_args={"host": "127.0.0.1", "port": "19530"},
    collection_name="contract",
)


In [6]:
llm = ChatOpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever(), 
    return_source_documents=True,
)

In [8]:
# testing it out

question = "Who are the parties to the Agreement and what are their defined names?"
result = qa_chain({"query": question})
result["result"]

'The parties to the Agreement are Cloud Investments Ltd. and Advisor. Their defined names are Silvan Joseph for Cloud Investments Ltd. and Jack Robinson for the Advisor.'

In [13]:
eval_questions = [
    "Who are the parties to the Agreement and what are their defined names?",
    "What is the termination notice?",
    " What are the payments to the Advisor under the Agreement?",
    "Can the Agreement or any of its obligations be assigned?",
    "Who owns the IP?",
    "Is there a non-compete obligation to the Advisor?",
    "Can the Advisor charge for meal time?",
    "In which street does the Advisor live?",
    "Is the Advisor entitled to social benefits?",
    "What happens if the Advisor claims compensation based on employment relationship with the Company?",
]

eval_answers = [
    "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)",
    "According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement.",
    " According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance.",
    "1. Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9 the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking.",
    "According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company.",
    "Yes. During the term of engagement with the Company and for a period of 12 months thereafter.",
    "No. See Section 6.1, Billable Hour doesn’t include meals or travel time.",
    "1 Rabin st, Tel Aviv, Israel",
    "No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company.",
    " If the Advisor is determined to be an employee of the Company by a governmental authority, payments to the Advisor will be retroactively reduced so that 60% constitutes salary payments and 40% constitutes payment for statutory rights and benefits. The Company may offset any amounts due to the Advisor from any amounts payable under the Agreement. The Advisor must indemnify the Company for any losses or expenses incurred if an employer/employee relationship is determined to exist.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

IndexError: list index out of range